<a href="https://colab.research.google.com/github/kainzc/Big-Data1/blob/main/3_days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install mrjob
!pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 KB 6.4 MB/s eta 0:00:00


In [2]:
from google.colab import files

# upload dataset
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving nashville-tweets-2019-01-28 to nashville-tweets-2019-01-28
User uploaded file "nashville-tweets-2019-01-28" with length 25039251 bytes


In [4]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 8.1 MB/s eta 0:00:00


In [3]:
# upload mrjob.conf under current directory(/content/mrjob.conf)
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving mrjob.conf to mrjob.conf
User uploaded file "mrjob.conf" with length 651 bytes


In [18]:
%%file 3_days.py
#write the top pragma string to save this file in local colab drive as 3_days.py
from mrjob.job import MRJob
import json

'''
Count the tweets per day.

See http://mike.teczno.com/notes/streaming-data-from-twitter.html for parsing info.
Get the screen name by accessing tweet['user']['screen_name']

Look at tweet['created_at'] for datetime of creation. Just use the first word in the date to get the day.


'''


class DaysTweets(MRJob):
    def mapper(self, _, line):
        try:
            tweet = json.loads(line)
            day = tweet['created_at'].split()[0] # get the first word of the date for day
            yield day, 1
        except Exception as e:
            pass

    def reducer(self, key, counts):
        total_count = sum(counts)
        yield key, total_count

if __name__ == '__main__':
    DaysTweets.run()

Overwriting 3_days.py


## Step 1. Run MR program locally(Colab)

In [13]:
#write the command to run it first to check for error
%%time
!python 3_days.py -r local nashville-tweets-2019-01-28 > 3_days.out

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/3_days.root.20230402.201702.517085
Running step 1 of 1...
job output is in /tmp/3_days.root.20230402.201702.517085/output
Streaming final output from /tmp/3_days.root.20230402.201702.517085/output...
Removing temp directory /tmp/3_days.root.20230402.201702.517085...
CPU times: user 28.2 ms, sys: 9.09 ms, total: 37.2 ms
Wall time: 2.13 s


In [14]:
!cat 3_days.out

"Sun"	6294


## Step 2. Run MR program on AWS EMR

### Please copy nashville-tweets-2019-01-28 into your S3 bucket. Note that if you already uploaded the data in previous step you can just use the same url here.

In [15]:
%%time
!python 3_days.py -r emr s3://coleq3/nashville-tweets-2019-01-28 --cloud-tmp-dir=s3://coleq3/tmp/ --cluster-id=j-1J06CIUMKDZ7L --conf-path mrjob.conf > 3_days.out

Creating temp directory /tmp/3_days.root.20230402.205313.086204
uploading working dir files to s3://coleq3/tmp/3_days.root.20230402.205313.086204/files/wd...
Copying other local files to s3://coleq3/tmp/3_days.root.20230402.205313.086204/files/
Adding our job to existing cluster j-1J06CIUMKDZ7L
  master node is ec2-54-88-13-76.compute-1.amazonaws.com
Waiting for Step 1 of 1 (s-2IZCDMOTFS0HY) to complete...
  PENDING (cluster is RUNNING: Running step)
  RUNNING for 0:00:51
  RUNNING for 0:01:21
  COMPLETED
Attempting to fetch counters from logs...
Looking for step log in /mnt/var/log/hadoop/steps/s-2IZCDMOTFS0HY on ec2-54-88-13-76.compute-1.amazonaws.com...
Waiting 10 minutes for logs to transfer to S3... (ctrl-c to skip)
Looking for step log in s3://coleq3/tmp/j-1J06CIUMKDZ7L/steps/s-2IZCDMOTFS0HY...
Looking for step log in /mnt/var/log/hadoop/steps/s-2IZCDMOTFS0HY on ec2-54-88-13-76.compute-1.amazonaws.com...
Looking for step log in s3://coleq3/tmp/j-1J06CIUMKDZ7L/steps/s-2IZCDMOTFS0H

In [16]:
!cat 3_days.out

"Sun"	6294


In [17]:
f = open('3_days.out')
lines = f.readlines()
f.close()
for line in lines:
  if line[:-1] in ['"Sun"\t6294']:
    print("passed")
  else:
    assert False

passed
